# **Loading and Cleaning the Dataset**

In [2]:
# IMPORT LIBRARIES
# I’m starting with the basic data analysis libraries
import pandas as pd
import numpy as np

# STEP 2: LOAD THE DATA
# I’m loading the transit dataset that simulates Lagos bus operations
df = pd.read_csv("lagos_transit_data.csv")

# Let’s preview the first few rows to understand the structure
print(df.head())


     Trip_ID Route_ID Bus_Stop_ID    Scheduled_Arrival       Actual_Arrival  \
0  TRIP00001  LAG-101         CMS  2024-01-01 14:23:00  2024-01-01 14:23:00   
1  TRIP00002  LAG-101   Ojuelegba  2024-01-02 01:09:00  2024-01-02 01:19:00   
2  TRIP00003  LAG-101       Ikeja  2024-01-01 12:56:00  2024-01-01 12:56:00   
3  TRIP00004  LAG-102       Lekki  2024-01-01 12:31:00  2024-01-01 12:31:00   
4  TRIP00005  LAG-102       Lekki  2024-01-01 16:36:00  2024-01-01 16:38:00   

   Delay_Minutes  Passenger_Count Day_of_Week Weather_Condition  Latitude  \
0              0               23     Tuesday             Rainy  6.457299   
1             10               44    Thursday             Sunny  6.535195   
2              0               31      Friday             Sunny  6.462096   
3              0               32    Thursday             Foggy  6.505590   
4              2               33   Wednesday             Rainy  6.494113   

   Longitude     Bus_Type  
0   3.339874     Standard  
1   3.

In [7]:
# I want to perform time-series analysis later, so I need to convert the date columns properly
df['Scheduled_Arrival'] = pd.to_datetime(df['Scheduled_Arrival'])
df['Actual_Arrival'] = pd.to_datetime(df['Actual_Arrival'])

# I’ll now extract components like hour, day of week, and month for pattern analysis
df['Hour'] = df['Scheduled_Arrival'].dt.hour
df['Month'] = df['Scheduled_Arrival'].dt.month
df['Weekday'] = df['Scheduled_Arrival'].dt.day_name()


In [8]:
# I want to know if a trip was delayed or not (binary)
df['Delay_Flag'] = df['Delay_Minutes'].apply(lambda x: 1 if x > 0 else 0)

# I’ll also classify delays into buckets to see if we’re dealing with short vs long delays
def categorize_delay(minutes):
    if minutes == 0:
        return 'No Delay'
    elif minutes <= 5:
        return 'Short'
    elif minutes <= 10:
        return 'Medium'
    else:
        return 'Long'

df['Delay_Category'] = df['Delay_Minutes'].apply(categorize_delay)


In [9]:
# Just a quick check to ensure our dataset is clean and doesn't contain missing or duplicated data
print("Missing values:\n", df.isnull().sum())
print("Duplicate rows:", df.duplicated().sum())


Missing values:
 Trip_ID              0
Route_ID             0
Bus_Stop_ID          0
Scheduled_Arrival    0
Actual_Arrival       0
Delay_Minutes        0
Passenger_Count      0
Day_of_Week          0
Weather_Condition    0
Latitude             0
Longitude            0
Bus_Type             0
Hour                 0
Month                0
Weekday              0
Delay_Flag           0
Delay_Category       0
dtype: int64
Duplicate rows: 0


In [10]:
# If I need to use this version for Power BI or future use, I’ll export it
df.to_csv("lagos_transit_cleaned.csv", index=False)


# **Exploratory Data Analysis (EDA)**

In [11]:
# Summary Statistics
# I’ll start by checking the general distribution of delay and passenger data.
print("Summary of Delay (in minutes):")
print(df['Delay_Minutes'].describe())

print("\nSummary of Passenger Count:")
print(df['Passenger_Count'].describe())


Summary of Delay (in minutes):
count    2000.000000
mean        3.434000
std         5.715503
min         0.000000
25%         0.000000
50%         0.000000
75%         5.000000
max        20.000000
Name: Delay_Minutes, dtype: float64

Summary of Passenger Count:
count    2000.000000
mean       38.859000
std        14.725778
min         0.000000
25%        29.000000
50%        39.000000
75%        49.000000
max        86.000000
Name: Passenger_Count, dtype: float64


In [12]:
# Most Common Delay Categories
# Let’s see how delays are distributed across categories: No Delay, Short, Medium, Long
delay_counts = df['Delay_Category'].value_counts()
print("\nDelay Category Distribution:")
print(delay_counts)



Delay Category Distribution:
Delay_Category
No Delay    1220
Short        379
Medium       204
Long         197
Name: count, dtype: int64


In [ ]:
# Average Delay by Hour of Day
# I want to know what time of day delays are most frequent — this can inform scheduling changes.
avg_delay_by_hour = df.groupby('Hour')['Delay_Minutes'].mean().sort_index()
print("\nAverage Delay by Hour of Day:")
print(avg_delay_by_hour)


In [14]:
# Average Delay by Bus Type
# I suspect bus size/type might affect delay rates due to speed or capacity. Let's check.
avg_delay_by_bus_type = df.groupby('Bus_Type')['Delay_Minutes'].mean()
print("\nAverage Delay by Bus Type:")
print(avg_delay_by_bus_type)



Average Delay by Bus Type:
Bus_Type
Articulated    3.570336
Mini           3.338192
Standard       3.398485
Name: Delay_Minutes, dtype: float64


In [15]:
# Impact of Weather on Delays
# Weather is likely a factor in delays, especially rain or fog.
avg_delay_by_weather = df.groupby('Weather_Condition')['Delay_Minutes'].mean()
print("\nAverage Delay by Weather Condition:")
print(avg_delay_by_weather)



Average Delay by Weather Condition:
Weather_Condition
Cloudy    3.129482
Foggy     3.730382
Rainy     3.666667
Sunny     3.217391
Name: Delay_Minutes, dtype: float64


In [16]:
# Route Performance Overview
# I want to identify which routes are most delayed overall.
route_delay = df.groupby('Route_ID')['Delay_Minutes'].mean().sort_values(ascending=False)
print("\nAverage Delay by Route:")
print(route_delay)



Average Delay by Route:
Route_ID
LAG-201    3.542787
LAG-301    3.482100
LAG-101    3.474531
LAG-202    3.410891
LAG-102    3.255696
Name: Delay_Minutes, dtype: float64
